# Práctica del módulo NLP

In [1]:
import pandas as pd
import numpy as np

import glob
import re

from keras.models import Sequential
from keras.layers import Dense, LSTM

Using TensorFlow backend.
C:\ProgramData\Anaconda3\envs\tensorflow-gpu\lib\site-packages\tensorflow\python\framework\dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\ProgramData\Anaconda3\envs\tensorflow-gpu\lib\site-packages\tensorflow\python\framework\dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\ProgramData\Anaconda3\envs\tensorflow-gpu\lib\site-packages\tensorflow\python\framework\dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\ProgramData\Anaco

Cargo todos los twitts y los concateno en un único dataframe

In [2]:
all_files = glob.glob('twitter_corpus/*.csv')
li = []

for filename in all_files:
    df = pd.read_csv(filename, index_col=None, header=0)
    li.append(df)

corpus = pd.concat(li, axis=0, ignore_index=True)
corpus.head()

,Unnamed: 0,date,id,link,retweet,text,author
0,0,Oct 4,783396985093193728,/missyscheng/status/783396985093193728,False,#DataScience Basics: #DataMining vs. #Statisti...,various
1,1,Oct 4,783381842024103936,/EXASOLAG/status/783381842024103936,False,How to Become a #Data Scientist – Part 1: http...,various
2,2,Oct 4,783433625723252736,/TarasNovak/status/783433625723252736,False,@jesterxl @kdnuggets or just go with @tableau :),various
3,3,Oct 4,783428740453982208,/kdnuggets/status/783428740453982208,False,#Boston U. Online MS in Applied #Business #Ana...,various
4,4,1h1 hour ago,787052623291641856,/kdnuggets/status/787052623291641856,False,#ICYMI Still Searching for ROI in #BigData Ana...,various


# 3. Generación de Lenguaje Natural

Voy a convertir todo el texto de los twits en un único documento plano de texto.

In [3]:
full_corpus = "\n\n".join(re.sub('[^a-zA-Z0-9!?\',.:"]+', '', tweet) for tweet in corpus.sample(25000).text)

Genero un diccionario de caracteres y dos conversores que transformen respectivamente cada caracter en su número correspondiente y a la inversa

In [4]:
chars = sorted(list(set(full_corpus)))
char_to_index = dict((c, i) for i, c in enumerate(chars))
index_to_char = dict((i, c) for i, c in enumerate(chars))

Genero los chunks de texto que se van a utilizar para entrenar el modelo

In [5]:
# Limit the sentences length to chunks of seq_length characters
seq_length = 40

# Spaces between sentences
step = 1

# List to store the sentences
sentences = []

# List to store the next character to be predicted for each sentence
next_chars = []

for i in range(0, len(full_corpus)-seq_length, step):
    sentences.append(full_corpus[i:i+seq_length])
    next_chars.append(full_corpus[i+seq_length])
    
print('num training examples: {}'.format(len(sentences)))

num training examples: 2462049


Genero un tensor de frases con cada carácter representado en un vector one-hot

In [ ]:
# Create one-hot-encoded vectors
X = np.zeros((len(sentences), seq_length, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)

for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        X[i, t, char_to_index[char]] = 1
    y[i, char_to_index[next_chars[i]]] = 1

Construyo el modelo y lo entreno

In [ ]:
model = Sequential()
model.add(LSTM(128, input_shape=(seq_length, len(chars))))
model.add(Dense(len(chars), activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer=RMSprop(lr=0.01), metrics=['accuracy'])

history = model.fit(X, y, validation_split=0.05, batch_size=128, epochs=50, shuffle=True).history

Para testear el modelo, lo mejor es ponerlo a prueba. Ya que no nos interesa tanto el accuracy como el hecho de que sea capaz de generar un texto coherente.

In [ ]:
example_sentence = "I've seen a red unicorn fighting with a giant purple dinosaur. I was so scared that I suddenly woke up.\n\n"

In [ ]:
def prepare_input(text):
    x = np.zeros((1, seq_length, len(chars)))

    for t, char in enumerate(text):
        x[0, t, char_indices[char]] = 1.
        
    return x

In [ ]:
def sample(preds):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds)
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    
    return heapq.nlargest(1, range(len(preds)), preds.take)

In [ ]:
def predict_completion(text):
    for _ in range(200):
        x = prepare_input(text[-seq_length:])
        preds = model.predict(x, verbose=0)[0]
        next_index = sample(preds)
        next_char = index_to_char[next_index]

        text += next_char
        
    return text

In [ ]:
predict_completion(prepare_input(example_sentence))

¿Y si intento generar texto usando un modelo preentrenado para generar poesía?